In [18]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%autoreload 2

In [20]:
import bittensor
import os
import torch
import torch.multiprocessing as mp 
import time
from loguru import logger
from termcolor import colored
import nest_asyncio 
nest_asyncio.apply()
logger.remove()

# **Init Bittensor** <a class="anchor" id="Init-Bittensor"></a>

## **Wallet** <a class="anchor" id="Wallet"></a>

In [21]:
# WALLET: Holds keys to run your miner and control funds.

# *****
# IMPORTANT: Store the mnemonic for **both** your hot and coldkey 
# you will need these to recover your keys if you forget your password or lose access to this machine.
# ******

# Fill in below to name your wallet and keys.
YOUR_WALLET_NAME = 'colab'
YOUR_HOTKEY_NAME = 'colab_hot'

# Fill in below if your need to regenerate your keys.
use_mnemonic = False # Set to true for key regeneration.
coldkey_mnemonic = "<to be filled>".split(' ')
hotkey_mnemonic = "<to be filled>".split(' ')

# Create the wallet object.
wallet = bittensor.Wallet(
    path = "~/.bittensor/wallets/",
    name = YOUR_WALLET_NAME,
    hotkey = YOUR_HOTKEY_NAME
)

# Optionally regens/creates your wallet keys.
if not wallet.has_coldkeypub:
    if use_mnemonic:
        wallet.regenerate_coldkey(mnemonic = coldkey_mnemonic, use_password = True)
    else:
        wallet.create_new_coldkey(n_words = 12, use_password = True )
if not wallet.has_hotkey:
    if use_mnemonic:
        wallet.regenerate_hotkey(mnemonic = hotkey_mnemonic)
    else:
        wallet.create_new_hotkey(n_words = 12)

# Assert before continuing
assert wallet.has_hotkey
assert wallet.has_coldkeypub

Loaded coldkey.pub: 0x3c9cd1679888e5660b0c8e4b8a17a1719c0cb7f05b5c624a856b421b52290515
Loaded hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545


## **Config** <a class="anchor" id="Config"></a>

In [22]:
# Uncomment to print all config items: 
# bittensor.help()
config = bittensor.Neuron.default_config()
config.neuron.multiprocessing = False
config.neuron.debug = True
config.receptor.do_backoff = False
config.receptor.timeout = 2
config.subtensor.network = 'kusanagi'

In [23]:
# Init bittensor components
# NOTE: Running this call recreates bittensor objects.
bittensor.init( with_config = config, with_wallet = wallet )
print ( bittensor.Config.toString( bittensor.config ) )


axon:
  backward_processing_timeout: 5
  external_ip: 190.119.146.134
  external_port: 8091
  forward_processing_timeout: 5
  local_ip: 127.0.0.1
  local_port: 8091
  max_workers: 10
  use_upnpc: false
metagraph:
  stale_emit_filter: -1
neuron:
  debug: true
  modality: 0
  multiprocessing: false
receptor:
  do_backoff: false
  max_backoff: 100
  pass_gradients: true
  timeout: 2
subtensor:
  chain_endpoint: null
  network: kusanagi
wallet:
  hotkey: colab_hot
  name: colab
  path: ~/.bittensor/wallets/



## **Metagraph** <a class="anchor" id="Metagraph"></a> 
**a.k.a the chain-state as a torch object**

In [38]:
# Sync chain information into the metagraph
# NOTE: expensive command.
bittensor.metagraph.sync()


Syncing metagraph:
Successfully connected to kusanagi endpoint: 157.230.11.1:9944
...............................................................................................................................................................................................................................................................................................

In [43]:
chain_block =  bittensor.metagraph.block()
uids_on_chain = bittensor.metagraph.uids()
n_neurons = torch.max(bittensor.metagraph.uids())
n_online = torch.numel(
  bittensor.metagraph.uids()[ 
    torch.where(
      bittensor.metagraph.block() - bittensor.metagraph.lastemit() < 100
    )
  ]
)
print ('The chain is at block: {}\n'.format(bittensor.metagraph.block()))
print ('There are {} neurons on the network\n'.format(n_neurons))
print ('These are their uids: {}\n'.format(bittensor.metagraph.uids()))
print ('{} neurons have emitted in the last 100 blocks\n'.format(n_online))
print ('{} Tao is staked'.format(torch.sum(bittensor.metagraph.S())))

The chain is at block: 820060

There are 285 neurons on the network

These are their uids: tensor([168, 116,  95, 113, 142, 280,  91, 195, 256,  22, 278, 190,  11,  71,
        140,   9, 107, 165, 222, 274, 258, 152,  44,  20, 205, 228,  45, 106,
        153,  67, 268,  84,  69, 174,  15,   2, 111, 191, 237, 130, 225, 129,
         98, 125, 218, 262, 144, 249, 155,  75,  38,  25, 212, 214, 124, 261,
         42, 163, 181, 182, 240, 171,  43,  34, 243, 197, 272, 217, 120, 164,
        255,   5, 200,  85, 267, 187, 244,   3, 242,  64,  77, 151, 202, 234,
        281, 170, 149, 189,  92,  65,  60, 241, 232, 178, 169, 166,  14, 257,
        159, 104,  83, 136,   8,   0, 137, 177,  76, 275, 247, 279,  90, 115,
         57,   6,  58, 157, 105,  30, 265, 236, 215, 253,  93, 114,  40, 186,
         48, 172, 194,  13, 226,  18,  72, 269, 260, 176, 204, 223, 110, 213,
        250, 282,  10,  26, 231,  68, 266, 284, 185,  50,  63,  78,  94, 206,
         79, 101, 122,  88, 162, 210, 134, 148, 179

## **Subscription** <a class="anchor" id="Subscription"></a>

**a.k.a advertising your self to the chain**

In [44]:
# Subscribe our endpoint to the chain.
bittensor.subtensor.subscribe(
    config.axon.external_ip, 
    config.axon.external_port,
    config.neuron.modality,
    wallet.coldkeypub,
    wait_for_finalization = True,
    timeout = 4 * bittensor.__blocktime__,
)

Successfully connected to kusanagi endpoint: 157.230.3.108:9944
Already subscribed with [ip: 190.119.146.134, port: 8091, modality: 0, coldkey: 0x3c9cd1679888e5660b0c8e4b8a17a1719c0cb7f05b5c624a856b421b52290515]


True

In [48]:
uid = bittensor.metagraph.uid_for_pubkey(wallet.hotkey.public_key)
index = bittensor.metagraph.index_for_uid(uid)
stake = bittensor.metagraph.S()[index]
rank = bittensor.metagraph.R()[index]
incentive = bittensor.metagraph.I()[index]
neuron = bittensor.metagraph.neurons()[index]
position = (torch.argsort(bittensor.metagraph.R(), dim=0) == index).nonzero(as_tuple=True)[0].item()
wallet_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
print ('Your network uid is {} for hotkey public key {}\n'.format(uid, wallet.hotkey.public_key))
print ('Your are subscribed at endpoint {}:{} for modality {}\n'.format(neuron.address, neuron.port, neuron.modality))
print ('You are staking \u03C4{} \n\nYou have a network rank of \u03C4{} for position {}/{}\n\nand are attaining incentive: \u03C4{}/block'.format(stake, rank, position, n_neurons, incentive))


Your network uid is 168 for hotkey public key 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545

Your are subscribed at endpoint 190.119.146.134:8091 for modality 0

You are staking τ113.33551025390625 

You have a network rank of τ580.676513671875 for position 123/285

and are attaining incentive: τ0.0011048010783270001/block


# **Nucleus**  <a class="anchor" id="Nucleus"></a>
**a.k.a your unique machine learning model**

## **GPT2-Nucleus** <a class="anchor" id="GPT2-Nucleus"></a>

In [78]:
import torch.nn as nn
import torch.nn.functional as F
from types import SimpleNamespace
from transformers import GPT2Config, GPT2Model

class GPT2Nucleus(torch.nn.Module):
    # A simple as it gets Bittensor nucleus using a GPT2 kernel
    def __init__(self):
        super().__init__()
        huggingface_config = GPT2Config( vocab_size=bittensor.__vocab_size__, n_embd=bittensor.__network_dim__, n_layer=2, n_head=1, n_inner=8 )
        self.transformer = GPT2Model(huggingface_config)
        self.hidden_layer = nn.Linear( bittensor.__network_dim__, bittensor.__network_dim__ )
        self.target_layer = nn.Linear( bittensor.__network_dim__, bittensor.__vocab_size__, bias=False )
        self.loss_fct = nn.CrossEntropyLoss()
        
        # The scores you learn for other neurons in the network.
        # NOTE: This needs to be updated every time you re-sync your metagraph.
        self.row_weights = torch.rand( bittensor.metagraph.n(), requires_grad=True)

    def local_forward(self, inputs: torch.LongTensor):
        # Runs only the local part of the model.

        # To be filled.
        output = SimpleNamespace()

        # Apply our GPT transformer model.
        # local_context.shape = [ batch_size, sequence_len, network_dim ]
        output.local_context = self.transformer(input_ids=inputs, return_dict=True).last_hidden_state

        # Apply our dense layer and project it onto our target layer.
        # local_hidden.shape = [ batch_size, sequence_len, network_dim ]
        output.local_hidden = self.hidden_layer( output.local_context )

        # Project to our target dimension.
        # local_targets.shape = [ batch_size, sequence_len, vocab_size ]
        output.local_targets = self.target_layer( output.local_hidden )

        # Compute LM-loss 
        shift_targets = output.local_targets[..., :-1, :].contiguous()
        shift_inputs = inputs[..., 1:].contiguous()
        output.local_loss = self.loss_fct(shift_targets.view(-1, shift_targets.size(-1)), shift_inputs.view(-1))

        return output

    def remote_forward(self, inputs: torch.LongTensor, n_to_query:int = 10):
        # Runs the model with calls on the network.

        # Run the local model.
        output = self.local_forward( inputs )

        # Select peers to query based on weights + random pertubation.
        gamma = 0.9
        output.query_weights, output.query_indices = torch.topk(
            self.row_weights + torch.rand_like(self.row_weights) * gamma, 
            n_to_query
        )
        neurons_to_call = [ bittensor.metagraph.neurons()[idx] for idx in output.query_indices.tolist() ]
        inputs_to_send = [ inputs for _ in output.query_indices.tolist() ]
        output.query_uids = [bittensor.metagraph.uid_for_pubkey(neuron.public_key) for neuron in neurons_to_call]

        # Make network calls and then weight-join the responses.
        output.codes, responses = bittensor.forward_text( 
            neurons = neurons_to_call, 
            inputs = inputs_to_send
        )
        stacked_responses = torch.stack( responses, dim=2 )
        output.remote_context = torch.matmul( torch.transpose( stacked_responses, dim0=2, dim1=3), output.query_weights)

        # Compute the distillation loss between the local and remote context
        output.distillation_loss = F.mse_loss( output.local_context, output.remote_context.detach() )

        # Apply the hidden dense layer to the context.
        output.remote_hidden = self.hidden_layer( output.remote_context )

        # Project to our target dimension.
        output.remote_targets = self.target_layer( output.remote_hidden )

        # Compute our loss against our remote context.
        shift_targets = output.remote_targets[..., :-1, :].contiguous()
        shift_inputs = inputs[..., 1:].contiguous()
        output.remote_loss = self.loss_fct(shift_targets.view(-1, shift_targets.size(-1)), shift_inputs.view(-1))

        return output

In [79]:
# Your nucleus
model = GPT2Nucleus()

## **Test-Nucleus** <a class="anchor" id="Test-Nucleus"></a>

In [80]:
# Test remote forward call.
inputs = torch.tensor([bittensor.__tokenizer__()('the cat')['input_ids']])
output = model.remote_forward( inputs, n_to_query=20)
loss = output.local_loss + output.remote_loss + output.distillation_loss
loss.backward() #
print ('The distillation loss between your local and remote context is  {}\n'.format(output.distillation_loss))
print ('The loss with respect to your local context and the targets is {}\n'.format(output.local_loss))
print ('The loss with respect to your remote context and the targets is {}\n'.format(output.remote_loss))
print ('You queried {} remote neurons with\n\nuids\n {}\n\nresponse codes\n {}\n'.format(torch.numel(output.codes), output.query_uids, output.codes.tolist()))
print ('and gradients w.r.t your row weights\n', [float('{:0.3f}'.format(model.row_weights.grad[idx].item())) for idx in output.query_indices.tolist()])


The distillation loss between your local and remote context is  12.811713218688965

The loss with respect to your local context and the targets is 10.775256156921387

The loss with respect to your remote context and the targets is 12.333410263061523

You queried 20 remote neurons with

uids
 [170, 49, 42, 255, 103, 219, 99, 238, 37, 30, 237, 206, 8, 73, 143, 284, 173, 258, 207, 105]

response codes
 [0, 0, 0, 3, 1, 0, 0, 3, 3, 0, 1, 0, 17, 0, 1, 1, 0, 3, 3, 3]

and gradients w.r.t your row weights
 [0.06, -0.011, 0.126, 0.0, 0.0, 0.411, -0.171, 0.0, 0.0, 0.517, 0.0, -0.016, 0.0, 0.353, 0.0, 0.0, 0.038, 0.0, 0.0, 0.0]


# **Training** <a class="anchor" id="Training"></a>

## **Training-Loop** <a class="anchor" id="Training-Loop"></a>

In [99]:
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from loguru import logger
import random
from datasets import load_dataset
import time
import os

# ---- Dataset ---- 
dataset = load_dataset('ag_news')['train']
def nextbatch(data, batch_size, tokenizer):
    """ Returns a random batch of sentences from text dataset.
    """
    batch_text = []
    for _ in range(batch_size):
        batch_text.append(data[random.randint(0, len(data))]['text'])
    batch_inputs = tokenizer(batch_text, return_tensors='pt', padding=True, truncation=True)['input_ids']
    return batch_inputs

# --- Training Logger ----
logger.remove()
training_log_dir = os.path.expanduser('~/logs/training.log')
logger.add(training_log_dir, filter=lambda record: "training" in record["extra"], enqueue=True, backtrace=True, diagnose=True, rotation="500 MB")
def show_training_logs(length: int = 25):
    ! tail -n $length $training_log_dir

# ---- Tokenizer ----
# For encoding text inputs.
tokenizer = bittensor.__tokenizer__()

# ---- Optimizer ----
# For applying your gradient steps to the local model.
optimizer = torch.optim.SGD( model.parameters(), lr = 0.1, momentum = 0.99 )

# ---- Training Loop -----
def train( 
        stop_training: mp.Event,
    ):
    # ---- Loop until event is set ----
    training_step = 0
    batch_size = 5
    logger.bind(training=True).info('Loop starting... ')
    while not stop_training.is_set():
        try:
            optimizer.zero_grad() # Zeros out gradients for next accummulation

            # ---- Forward pass ----
            inputs = nextbatch( dataset, batch_size, tokenizer )
            outputs = model.remote_forward( inputs )

            # ---- Backward pass ----
            loss = outputs.local_loss + outputs.remote_loss + outputs.distillation_loss
            loss.backward() # Accumulates gradients on the model.
            clip_grad_norm_(model.parameters(), 0.8) # clip model gradients
            optimizer.step() # Applies accumulated gradients.

            # ---- Step logs ----
            logger.bind(training=True).info('->\nuids:{}\ncodes:{}\nweights:{}\ngrads:{}', 
                  outputs.query_uids, 
                  outputs.codes.tolist(), 
                  [float('{:0.3f}'.format(x)) for x in outputs.query_weights.tolist()],
                  [float('{:0.3f}'.format(model.row_weights.grad[idx].item())) for idx in outputs.query_indices.tolist()])      
            logger.bind(training=True).info('gs:{} loss(local):{} loss(remote):{} loss(distill):{} dendrite:{}',
                  colored('{}'.format( training_step ), 'red'),
                  colored('{:.4f}'.format(outputs.local_loss.item()), 'green'),
                  colored('{:.4f}'.format(outputs.remote_loss.item()), 'blue'),
                  colored('{:.4f}'.format(outputs.distillation_loss.item()), 'red'),
                  bittensor.neuron.dendrite.toString())
            training_step += 1
        except Exception as e:
            logger.bind(training=True).exception("Training iteration exception.")
    logger.bind(training=True).complete()

Using custom data configuration default
Reusing dataset ag_news (/Users/const/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)


## **Training Thread Runners** <a class="anchor" id="Training-Thread-Runners"></a>

In [100]:
import threading
import torch.multiprocessing as mp 
import sys

join_timeout = 10

if 'quit_training' in locals():
    quit_training.set()
if 'training_thread' in locals() and training_thread.is_alive():
    training_thread.join( timeout = join_timeout )

quit_training = mp.Event()
training_thread = threading.Thread( target = train, args = (quit_training,),  name = 'training', daemon=True)

def stop_training():
    global quit_training
    global training_thread
    quit_training.set()
    if not training_thread.is_alive():
        return
    logger.bind(training=True).info("Joining...")
    training_thread.join( timeout = join_timeout )
    if not training_thread.is_alive():
        print ('Joined training thread',)
        logger.bind(training=True).info('Joined.')
    else:
        print ('Failed to join training thread')

def start_training():
    global quit_training
    global training_thread
    stop_training()
    quit_training = mp.Event()
    training_thread = threading.Thread( target = train, args = (quit_training,), name = 'training', daemon=True)
    training_thread.start()
    logger.bind(training=True).info("Started training.")
    print('new training thread:', training_thread)

In [103]:
start_training()

new training thread: <Thread(training, started daemon 123145557131264)>


In [102]:
print (training_thread.is_alive())

False


In [101]:
stop_training()

In [171]:
show_training_logs(50)

grads:[0.353, 0.0, 0.0, 0.0, 0.0, 0.0, 0.038, -0.011, 0.0, 0.0]
2021-03-16 17:21:36.508 | INFO     | __main__:train:69 - gs:550 loss(local):6.9424 loss(remote):6.4884 loss(distill):0.0270 dendrite:(0.445q/s|⤊ 687.3/⤋ 0.3kB/s)
2021-03-16 17:21:38.546 | INFO     | __main__:train:63 - ->
uids:[170, 103, 147, 70, 45, 42, 161, 67, 99, 32]
codes:[1, 1, 3, 1, 1, 1, 3, 1, 1, 3]
weights:[1.84, 1.834, 1.765, 1.744, 1.714, 1.711, 1.702, 1.648, 1.635, 1.624]
grads:[0.06, 0.0, 0.0, 0.0, 0.0, 0.126, 0.0, 0.0, -0.171, 0.0]
2021-03-16 17:21:38.547 | INFO     | __main__:train:69 - gs:551 loss(local):7.2448 loss(remote):6.8455 loss(distill):0.0222 dendrite:(0.445q/s|⤊ 687.0/⤋ 0.3kB/s)
2021-03-16 17:21:40.612 | INFO     | __main__:train:63 - ->
uids:[73, 278, 121, 222, 258, 32, 152, 224, 158, 276]
codes:[1, 3, 1, 1, 3, 3, 3, 3, 3, 3]
weights:[1.842, 1.787, 1.742, 1.725, 1.715, 1.714, 1.696, 1.656, 1.646, 1.633]
grads:[0.353, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2021-03-16 17:21:40.612 | INFO     

# **Serving** <a class="anchor" id="Serving"></a>

In [113]:
# Start the axon serving endpoint.
bittensor.axon.start()

## **Serving-Loop** <a class="anchor" id="Serving-Loop"></a>

In [114]:
# ---- Serving logger ----
serving_log_dir = os.path.expanduser('~/logs/serving.log')
logger.add(serving_log_dir, filter=lambda record: "serving" in record["extra"], enqueue=True, backtrace=True, diagnose=True, rotation="500 MB")
def show_serving_logs(length: int = 25):
    ! tail -n $length $serving_log_dir

# ---- Serving loop -----
def serve ( 
  stop_serving: mp.Event,
):

    # ---- Loop until event is set -----
    serving_step = 0
    logger.bind(serving=True).info('Serving thread started: ')
    while not stop_serving.is_set():

        # ---- Pull request ----
        logger.bind(serving=True).info('Axon:{}, waiting for query ... ', bittensor.axon.toString())
        pong, pubkey, inputs, modality = bittensor.axon.next_forward_item( timeout = 10.0 )

        # ---- Process request ----
        if None not in [ pong, pubkey, inputs, modality]:
            logger.bind(serving=True).info('Recieved Query: from:{}, inputs.shape:{}', pubkey, inputs.shape)
            try:          
                outputs = model.local_forward( inputs ).local_hidden
                pong.send( outputs.detach() )
                logger.bind(serving=True).info('Sent response: to:{}, outputs.shape:{}', pubkey, outputs.shape)

            except Exception as e:
                logger.bind(serving=True).exception('Error in forward process with error {}', e)
                continue

      # ---- Tensorboard ----
      #bittensor.neuron.axon.toTensorboard(serving_tensorboard, serving_step)

## **Serving Thread Runners** <a class="anchor" id="Serving-Thread-Runners"></a>

In [115]:
join_timeout = 10

if 'quit_serving' in locals():
    quit_serving.set()
if 'serving_thread' in locals() and serving_thread.is_alive():
    serving_thread.join( timeout = join_timeout )

quit_serving = mp.Event()
serving_thread = threading.Thread( target = serve, args = (quit_serving,),  name = 'serving', daemon=True)

def stop_serving():
    global quit_serving
    global serving_thread
    quit_serving.set()
    if serving_thread.is_alive():
        logger.bind(serving=True).info("Joining...")
        serving_thread.join( timeout = join_timeout )
    if not serving_thread.is_alive():
        print ('Joined serving thread',)
        logger.bind(serving=True).info('Joined.')
    else:
        print ('Failed to join serving thread')

def start_serving():
    global quit_serving
    global serving_thread
    stop_serving()
    quit_serving = mp.Event()
    serving_thread = threading.Thread( target = serve, args = (quit_serving,), name = 'serving', daemon=True)
    serving_thread.start()
    logger.bind(serving=True).info("Started serving.")
    print('new serving thread:', serving_thread)


In [117]:
start_serving()

Joined serving thread
new serving thread: <Thread(serving, started daemon 123146017439744)>


In [124]:
print (serving_thread.is_alive())

True


In [94]:
stop_serving()

Joined serving thread


In [170]:
show_serving_logs(25)

2021-03-16 17:17:48.043 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-03-16 17:17:58.048 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-03-16 17:18:08.053 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-03-16 17:18:18.058 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-03-16 17:18:28.060 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-03-16 17:18:38.064 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-03-16 17:18:48.067 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-03-16 17:18:58.069 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waiting for query ... 
2021-03-16 17:19:08.074 | INFO     | __main__:serve:18 - Axon:(0.000q/s|⤊ 0.0/⤋ 0.0kB/s), waitin

## **Test-Serving** <a class="anchor" id="Test-Serving"></a>

In [126]:
axon_endpoint = bittensor.proto.Neuron(
    address = bittensor.neuron.config.axon.external_ip,
    port = bittensor.neuron.config.axon.external_port,
    public_key = wallet.hotkey.public_key
)
start_time = time.time()
codes, responses = bittensor.forward_text( 
    neurons = [ axon_endpoint ],
    inputs = [ torch.tensor([[1]]) ]
)
end_time = time.time()
print(colored('Querying endpoint: {}:{}'.format(axon_endpoint.address, axon_endpoint.port), 'blue'))
if codes.item() == bittensor.proto.ReturnCode.Success:
    print(colored('Success', 'green'))
    print(colored('Response shape: {}'.format(responses[0].shape) , 'green'))
    print(colored('Query time: {}'.format(end_time - start_time) , 'green'))
else:
    print(colored('Failure with code: {}'.format(codes.item()), 'red'))
    print(colored('Ensure your axon is started with bittensor.axon.start()', 'red'))
    print(colored('Ensure your endpoint is accessable from the internet, perhaps behind your router\'s NAT?: {}', 'red'))


Querying endpoint: 190.119.146.134:8091
Failure with code: 1
Ensure your axon is started with bittensor.axon.start()
Ensure your endpoint is accessable from the internet, perhaps behind your router's NAT?: {}


# Weights

## Filtering Weights

In [166]:
# Get the trained weights from the chain.
weights_to_emit = model.row_weights.detach()

# Take topk
topk = 30
weights_to_emit, indices = torch.topk(weights_to_emit, topk)

# Normalize to 0,1
weights_to_emit = F.normalize(weights_to_emit, p = 1, dim = 0)

# Scatter back to size n.
weights_to_emit = torch.scatter(torch.zeros(bittensor.metagraph.n()), 0, indices, weights_to_emit)
print('Emitting weights: \n{}'.format(weights_to_emit))

Emitting weights: 
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0349, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0344, 0.0000, 0.0330, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0318, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0348, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0332, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0326, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0350, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.000

## **Setting Weights**

In [168]:
# Sets your incentive weights on the chain.
bittensor.metagraph.set_weights( weights = weights_to_emit )

AttributeError: 'NoneType' object has no attribute 'get'

In [169]:
your_uid = bittensor.metagraph.uids()[0]
print ('Your weights (encoded as uint32s) on the chain are: \n\n {} \n'.format(bittensor.subtensor.weight_vals_for_uid( your_uid )))

print ('For uids \n {}'.format(bittensor.subtensor.weight_uids_for_uid( your_uid )))

Your weights (encoded as uint32s) on the chain are: 

 [150009391, 147535183, 141842111, 136565071, 149446607, 142679455, 140086015, 150197407, 136175455, 145556895, 141021679, 146305919, 140856623, 145208111, 140773279, 142767551, 140623839, 139410175, 144121551, 147253727, 142157759, 146636943, 149784303, 137127039, 142540607, 140468831, 142150655, 135931647, 150579423, 139154044] 

For uids 
 [278, 222, 258, 67, 42, 255, 242, 170, 6, 30, 93, 173, 143, 147, 207, 158, 121, 37, 103, 277, 139, 49, 73, 161, 127, 276, 196, 252, 224, 55]


# **Transactions** <a class="anchor" id="Transactions"></a>

## **Unstaking-Funds** <a class="anchor" id="Unstaking-Funds"></a>

In [110]:
from bittensor.utils.balance import Balance
amount_tao = 0.1
amount = Balance.from_float( amount_tao )
print(colored("Sending Extrinsic: [Unstake: {} Tao from hotkey: {}]".format( amount.tao, wallet.hotkey.public_key) , 'blue'))
print ('waiting for finalization...')
result = bittensor.subtensor.unstake( amount, wallet.hotkey.public_key, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
    new_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
    new_stake = bittensor.subtensor.get_stake_for_uid( bittensor.metagraph.uid_for_pubkey(wallet.hotkey.public_key) )
    print(colored("Unstaked: {} Tao from hotkey: {} to coldkey.pub: {}".format( amount.tao, wallet.hotkey.public_key, wallet.coldkey.public_key ) , 'green'))
    print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
    print(colored("Your hotkey has new stake: {} Tao".format( new_stake.tao ) , 'green'))
else:
    print(colored("Unstaking transaction failed", 'red'))

Sending Extrinsic: [Unstake: 0.1 Tao from hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545]
waiting for finalization...


Enter password to unlock key:  ··············


decrypting key... (this may take a few moments)
Loaded coldkey: 0x3c9cd1679888e5660b0c8e4b8a17a1719c0cb7f05b5c624a856b421b52290515
Unstaked: 0.1 Tao from hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545 to coldkey.pub: 0x3c9cd1679888e5660b0c8e4b8a17a1719c0cb7f05b5c624a856b421b52290515
Your coldkey has new balance: 0.2 Tao
Your hotkey has new stake: 112.067646278 Tao


## **Staking-Funds** <a class="anchor" id="Staking-Funds"></a>

In [111]:
from bittensor.utils.balance import Balance 
amount_tao = 0.1
amount = Balance.from_float( amount_tao )
print(colored("Sending Extrinsic: [Stake: {} Tao to hotkey: {}]".format( amount.tao, wallet.hotkey.public_key) , 'blue'))
print ('waiting for finalization...')
result = bittensor.subtensor.add_stake( amount, wallet.hotkey.public_key, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
  new_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
  new_stake = bittensor.subtensor.get_stake_for_uid(bittensor.metagraph.uid_for_pubkey(wallet.hotkey.public_key))
  print(colored("Staked: {} Tao to hotkey: {} from coldkey.pub: {}".format( amount.tao, wallet.hotkey.public_key, wallet.coldkey.public_key ) , 'green'))
  print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
  print(colored("Your hotkey has new stake: {} Tao".format( new_stake.tao ) , 'green'))

else:
  print(colored("Staking transaction failed", 'red'))

Sending Extrinsic: [Stake: 0.1 Tao to hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545]
waiting for finalization...
Staked: 0.1 Tao to hotkey: 0x80cacfbdf7b155b39de22680a7cb14c61a8f95df702c92f5f142d25cca37c545 from coldkey.pub: 0x3c9cd1679888e5660b0c8e4b8a17a1719c0cb7f05b5c624a856b421b52290515
Your coldkey has new balance: 0.1 Tao
Your hotkey has new stake: 112.170295159 Tao


## **Transfering-Funds** <a class="anchor" id="Transfering-Funds"></a>

In [201]:
amount = 0.01
destination_public_key = wallet.coldkey.public_key
amount = Balance.from_float( amount )
balance = bittensor.subtensor.get_balance( wallet.coldkey.public_key )
if balance < amount:
    print(colored("Not enough balance ({}) to transfer {}".format(balance, amount), 'red'))
    quit()

print(colored("Requesting transfer of {} Tao, from coldkey.pub: {} to dest.pub: {}".format(amount.tao, wallet.coldkey.public_key, destination_public_key), 'blue'))
print("Waiting for finalization...",)
result = bittensor.subtensor.transfer(destination_public_key, amount, wait_for_finalization = True, timeout = bittensor.__blocktime__ * 5)
if result:
    print(colored("Transfer finalized with amount: {} Tao to dest: {} from coldkey.pub: {}".format(amount.tao, destination_public_key, wallet.coldkey.public_key), 'green'))
    new_balance = bittensor.subtensor.get_balance(wallet.coldkeypub)
    destination_balance = bittensor.subtensor.get_balance(destination_public_key)
    print(colored("Your coldkey has new balance: {} Tao".format( new_balance.tao ) , 'green'))
    print(colored("The destination has new balance: {} Tao".format( new_balance.tao ) , 'green'))
else:
    print(colored("Transfer failed", 'red'))

AttributeError: module 'bittensor' has no attribute 'utils'